In [14]:
from matplotlib import pyplot as plt         #импортируем библиотеки
from matplotlib.colors import ListedColormap

import numpy as np
import pandas as pd
from sklearn.linear_model import Perceptron as sklPerceptron
# для разделения df на train и test выборку
from sklearn.model_selection import train_test_split
# для метрик
from sklearn.metrics import roc_auc_score

In [4]:
df = pd.read_csv("Яблоки и груши.csv")

In [5]:
df.head()

,yellowness,symmetry,target
0,0.779427,0.257305,1.0
1,0.777005,0.015915,1.0
2,0.977092,0.304210,1.0
3,0.043032,0.140899,0.0
4,0.760433,0.193123,1.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   yellowness  1000 non-null   float64
 1   symmetry    1000 non-null   float64
 2   target      1000 non-null   float64
dtypes: float64(3)
memory usage: 23.6 KB


In [8]:
# разделение выборки на тестовую и учебную
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)
X_train = train_set.drop("target", axis = 1)
y_train = train_set["target"].copy()

X_test = test_set.drop("target", axis = 1)
y_test = test_set["target"].copy()

X_full = df.drop("target", axis = 1)
y_full = df["target"].copy()

In [11]:
sklPer = sklPerceptron()

In [12]:
sklPer.fit(X_train, y_train)

Perceptron()

In [13]:
y_test_pred = sklPer.predict(X_test)

In [15]:
roc_auc_score(y_test, y_test_pred)

0.9941860465116279

In [16]:
def Loss(y_pred, y):
    '''
    Рассчитаем среднеквадратичную ошибку
    '''
    y_pred = y_pred.reshape(-1,1)
    y = np.array(y).reshape(-1,1)
    return 0.5 * np.mean((y_pred - y) ** 2)


class Perceptron:
    def __init__(self, w=None, b=0):
        """
        :param: w -- вектор весов
        :param: b -- смещение
        """
        # Пока что мы не знаем размер матрицы X, а значит не знаем, сколько будет весов
        self.w = w
        self.b = b

    def activate(self, x):
        return np.array(x > 0, dtype=np.int64)

    def forward_pass(self, X):
        """
        Эта функция рассчитывает ответ перцептрона при предъявлении набора объектов
        :param: X -- матрица объектов размера (n, m), каждая строка - отдельный объект
        :return: вектор размера (n, 1) из нулей и единиц с ответами перцептрона
        """
        n = X.shape[0]
        y_pred = np.zeros((n, 1))  # y_pred == y_predicted - предсказанные классы
        y_pred = self.activate(X @ self.w.reshape(X.shape[1], 1) + self.b)
        return y_pred.reshape(-1, 1)

    def backward_pass(self, X, y, y_pred, learning_rate=0.005):
        """
        Обновляет значения весов перцептрона в соответствии с этим объектом
        :param: X -- матрица объектов размера (n, m)
                y -- вектор правильных ответов размера (n, 1)
                learning_rate - "скорость обучения" (символ alpha в формулах выше)
        В этом методе ничего возвращать не нужно, только правильно поменять веса
        с помощью градиентного спуска.
        """
        n = len(y)
        y = np.array(y).reshape(-1, 1)
        self.w = self.w - learning_rate * (X.T @ (y_pred - y) / n)
        self.b = self.b - learning_rate * np.mean(y_pred - y)

    def fit(self, X, y, num_epochs=300):
        """
        Спускаемся в минимум
        :param: X -- матрица объектов размера (n, m)
                y -- вектор правильных ответов размера (n, 1)
                num_epochs -- количество итераций обучения
        :return: losses -- вектор значений функции потерь
        """
        self.w = np.zeros((X.shape[1], 1))  # столбец (m, 1)
        self.b = 0  # смещение
        losses = []  # значения функции потерь на различных итерациях обновления весов

        for i in range(num_epochs):
            # предсказания с текущими весами
            y_pred = self.forward_pass(X)
            # считаем функцию потерь с текущими весами
            losses.append(Loss(y_pred, y))
            # обновляем веса в соответсвие с тем, где ошиблись раньше
            self.backward_pass(X, y, y_pred)

        return losses

In [19]:
perceptron = Perceptron()
losses = perceptron.fit(X_train.values, y_train.values.reshape((-1, 1)))
y_test_pred01 = perceptron.forward_pass(X_test).ravel()

In [20]:
roc_auc_score(y_test, y_test_pred01)

0.9941860465116279

In [21]:
roc_auc_score(y_test, y_test_pred)

0.9941860465116279

In [ ]:
# используя метрику roc auc можно понять что перцептрон написанный на практике и из библиотеки sklearn работают одинаково хорошо